In [83]:
# default_exp covidMap

In [84]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [85]:
#export
import folium
import pandas as pd
import io
import requests
import json
import os
import matplotlib.pyplot as plt
#import matplotlib.dates as mdates
#import matplotlib.pyplot as pyplot
import numpy as np
#import calendar
#import seaborn as sns
from datetime import datetime, timedelta

## Data

In [86]:
def saveDF(df,root,name,ext="csv"):
    directory = os.path.join("data",root)
    if not os.path.exists(directory):
        os.makedirs(directory)
    df.to_csv(os.path.join(directory, name + "." +  (datetime.now() - timedelta(days=1)).strftime("%Y.%m.%d") + "." + ext))

def getRemoteDF(url,name,ext="csv"):
    s=requests.get(surl).content
    df = pd.read_csv(io.StringIO(s.decode('utf-8')))
    saveDF(df=df,root=name,name=name,ext=ext)
    return df

In [87]:

def saveJson(jso,root,name,ext="json"):
    directory = os.path.join("data",root)
    if not os.path.exists(directory):
        os.makedirs(directory)
    theP = os.path.join(directory, name + "." +  (datetime.now() - timedelta(days=1)).strftime("%Y.%m.%d") + "." + ext)
    with open(theP, "w") as dataFile:
        json.dump(info, dataFile, indent=4, sort_keys=True)


def getRemoteJSON(url,name,ext="json"):
    res = requests.get(url)   
    info = res.json()
    saveJson(info,root=name,name=name,ext=ext)
    return info

def getStaticJSON(url,root,name,ext="json"):
    res = requests.get(url)   
    info = res.json()
    directory = os.path.join("data",root)
    
    if not os.path.exists(directory):
        os.makedirs(directory)
        
    theP = os.path.join(directory, name  + "." + ext)
    with open(theP, "w") as dataFile:
        json.dump(info, dataFile, indent=4, sort_keys=True)

    return info

In [88]:
#res = requests.get('https://data.sfgov.org/resource/favi-qct6.json')
#We turn the data into json. It would become dictionary in the Python.
#covid_current = res.json()

In [89]:
#x = covid_current[0]
#x
#https://data.sfgov.org/resource/f9wk-m4qb.json
#geozipCodes = getStaticJSON('https://data.sfgov.org/resource/f9wk-m4qb.json',root="sanFranZip",name="geozipCodes")   
#geozipCodes = res.json()
#geozipCodes

In [90]:
#names = [x["zip"] for x in geozipCodes]
#names[:5]
#geozipCodes[1]

In [91]:
surl = "https://data.sfgov.org/resource/favi-qct6.csv"
sanFranZip = getRemoteDF(surl,name="sanFranZip",ext="csv")
sanFranZip = sanFranZip.rename({"zip_code":"zip"},axis=1)
sanFranZip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   data_as_of           27 non-null     object 
 1   objectid             27 non-null     int64  
 2   zip                  27 non-null     int64  
 3   count                23 non-null     float64
 4   acs_population       27 non-null     int64  
 5   rate                 23 non-null     float64
 6   rate_groups          27 non-null     object 
 7   city_count           27 non-null     int64  
 8   city_acs_population  27 non-null     int64  
 9   city_rate            27 non-null     float64
 10  rate_diff            23 non-null     float64
 11  multipolygon         27 non-null     object 
dtypes: float64(4), int64(5), object(3)
memory usage: 2.7+ KB


In [92]:
#{"type":"Feature","id":"AL","properties":{"name":"Alabama"},"geometry":{"type":"Polygon","coordinates":[[[-87.359296,35.00118],[-85.606675,34.984749],[-85.431413,34.124869],[-85.184951,32.859696],[-85.069935,32.580372],[-84.960397,32.421541],[-85.004212,32.322956],[-84.889196,32.262709],[-85.058981,32.13674],[-85.053504,32.01077],[-85.141136,31.840985],[-85.042551,31.539753],[-85.113751,31.27686],[-85.004212,31.003013],[-85.497137,30.997536],[-87.600282,30.997536],[-87.633143,30.86609],[-87.408589,30.674397],[-87.446927,30.510088],[-87.37025,30.427934],[-87.518128,30.280057],[-87.655051,30.247195],[-87.90699,30.411504],[-87.934375,30.657966],[-88.011052,30.685351],[-88.10416,30.499135],[-88.137022,30.318396],[-88.394438,30.367688],[-88.471115,31.895754],[-88.241084,33.796253],[-88.098683,34.891641],[-88.202745,34.995703],[-87.359296,35.00118]]]}},

In [93]:
codes = [str(x) for x in sanFranZip.zip.unique()]

def clean(x):
    ret = {"id":x["zip"],"type":"Feature","properties":{"name":x["zip"]},"geometry":x["the_geom"]}

    return ret
    
geosanFran = {"type":"FeatureCollection","features":[ clean(x) for x in geozipCodes if x["zip"] in codes]}

#geosanFran

In [94]:
saveJson(geosanFran,root="sanFranZip",name='sanzip',ext="geojson")



In [95]:
sanFranZip.index = sanFranZip["zip"]
sanFranZip["zname"] = sanFranZip["zip"].apply( lambda x : str(x))
sanFranZip.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27 entries, 94121 to 94116
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   data_as_of           27 non-null     object 
 1   objectid             27 non-null     int64  
 2   zip                  27 non-null     int64  
 3   count                23 non-null     float64
 4   acs_population       27 non-null     int64  
 5   rate                 23 non-null     float64
 6   rate_groups          27 non-null     object 
 7   city_count           27 non-null     int64  
 8   city_acs_population  27 non-null     int64  
 9   city_rate            27 non-null     float64
 10  rate_diff            23 non-null     float64
 11  multipolygon         27 non-null     object 
 12  zname                27 non-null     object 
dtypes: float64(4), int64(5), object(4)
memory usage: 3.0+ KB


In [96]:
sanFranZip["zname"].unique()

array(['94121', '94123', '94158', '94107', '94118', '94105', '94114',
       '94133', '94132', '94112', '94117', '94127', '94102', '94124',
       '94109', '94134', '94129', '94103', '94108', '94115', '94130',
       '94110', '94122', '94131', '94111', '94104', '94116'], dtype=object)

In [97]:

SF_COORDINATES = (37.76, -122.45)

#https://data.sfgov.org/resource/tmnf-yvry.csv
#res = requests.get('https://data.sfgov.org/resource/favi-qct6.json')
#We turn the data into json. It would become dictionary in the Python.
#covid_current = res.json()

#crimedata = pd.read_csv('SFPD_Incidents_2015.csv')
 
# for speed purposes
MAX_RECORDS = 1000
  
# creation of the choropleth
map1 = folium.Map(location=SF_COORDINATES, zoom_start=12)
map1.choropleth(
    geo_data=geosanFran,
    data=sanFranZip,
    columns=['zname',"count" ],
    key_on='feature.properties.name',
    fill_color='YlGnBu'


)   

display(map1)

## census track

In [98]:
#https://data.sfgov.org/resource/tpyr-dvnc.geojson
info = getRemoteJSON('https://data.sfgov.org/resource/tpyr-dvnc.geojson',"rawZip",ext="geojson")


In [99]:
codes = [str(x) for x in sanFranZip.zip.unique()]

def clean(x):
    #ret = {"id":x["zip"],"type":"Feature","properties":{"name":x["zip"]},"geometry":x["the_geom"]}
    x['properties']["name"] = x['properties']['id']

    return x
#'crs':info['crs'],
geoCensus = {"type":info["type"],"features":[ clean(x) for x in info["features"] if x['properties']["area_type"] == "Census Tract"]}
len(info["features"]),len(geoCensus["features"])

(264, 195)

In [100]:
(datetime.now() - timedelta(days=1)).strftime("%Y.%m.%d") 

'2020.06.16'

In [101]:
saveJson(geoCensus,root="rawZip",name="census",ext="geojson")

In [102]:
features = ['rate', 'last_updated_at', 'id', 'area_type']
vals  = ['acs_population', 'count', 'deaths']
df = {}
for x in features:
    df[x] = []
    
for x in vals:
    df[x] = []
for y in geoCensus["features"]:
    for x in features:
        #print(x,y)
        df[x].append(y["properties"][x])
    for x in vals:
        #print(x,y)
        v = y["properties"][x]
        if v is None:
            df[x].append(0)
        else:
            df[x].append(int(v))
df = pd.DataFrame.from_dict(df).fillna(0)
#df.dropna()
#dead = df[df.deaths > 0]
#dead.head()
df["count"].unique()


array([  0,  16,  40,  13,  38,  10,  17,  70,  15,  14,  11,  21,  25,
        28,  54,  29,  18,  30,  12,  34, 129,  42,  91,  58,  53,  36,
        22,  32,  64,  46,  26,  23,  84,  24,  72,  52,  20])

In [103]:
communities_map = folium.Map(location=SF_COORDINATES, zoom_start=12)

map1.choropleth(
    geo_data=geoCensus,
    data=df,
    columns=['id',"count" ],
    key_on='feature.properties.name',
    fill_color='OrRd'


) 

# generate choropleth map 
choropleth = folium.Choropleth(
    geo_data=geoCensus,
    data=df,
    columns=['id',"count" ],
    key_on='feature.properties.name',
    fill_color='OrRd',
    #fill_color='RdYlGn',
    legend_name='Cases of Covid',
    highlight=True,
    threshold_scale=[0, 1, 16, 64, 128, 512],
    smooth_factor=0).add_to(communities_map)

# add labels indicating the name of the community
style_function = "font-size: 15px; font-weight: bold"
choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['name'], style=style_function, labels=False))

# create a layer control
folium.LayerControl().add_to(communities_map)

# display map
communities_map

In [104]:
"06075013500"

'06075013500'